<a href="https://colab.research.google.com/github/cnilsen/kcfp/blob/master/Demo_maptool_v0_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#King County Fish Passage Hydrology Tool

> Copyright (c) Geosyntec Consultants, 2019 <br>
<div>The copyright holders grant the freedom to copy, modify, convey, adapt, and/or redistribute this work under the terms of the
<a href="https://spdx.org/licenses/MPL-2.0.html">Mozilla Public Licesnse 2.0</a>

</div>

### **Note** 
Access to this the cloud tools in this notebook is restricted to members of the 
Stormwater Heatmap Google Users Group. 
Visit: https://groups.google.com/forum/#!forum/stormwaterheatmap to request access. 

### **Overview**
This tool supports King County's comprehensive inventory and assessment of assets' on watercourses to
determine if the assets are barriers to fish passage. King County is performing the assessment using
methodology outlined in the Washington Department of Fish and Wildlife's Fish Passage Inventory,
Assessment, and Prioritization Manual (WDFW, 2019).
The Level B assessment outlined in the WDFW
2019 Manual provides for a hydraulic analysis to determine if a culvert meets the velocity and depth
requirements for fish passage between low and high fish passage flows. Per the WDFW 2019 Manual:
For an instream feature to be considered a non-barrier, it should not obstruct upstream migration
at any time between the low and high fish passage flows at that location:

* Low fish passage flow is the 95% exceedance flow; and
* High fish passage flow is the 10% exceedance flow during the months of adult upstream migration.

### **Data Sources**

Data used in this tool has been provided from the Nature Conservancy's Stormwater Heatmap Project. Documentation on the Stormwater Heatmap is in progress. Below is a brief summary of data sources: 

* Digital Elevation Model - [USGS National Hydrography (NHD Plus High Resolution Raster Data)](https://www.usgs.gov/core-science-systems/ngp/national-hydrography/nhdplus-high-resolution)
* Soils - [USDA Gridded SURGO Database](https://www.nrcs.usda.gov/wps/portal/nrcs/detail/soils/home/?cid=nrcs142p2_053628)
* Slope - USGS National Elevation Dataset (https://ned.usgs.gov)
* Level B Sites - Spatial data provided by King County 
* Land cover and hydrology- The Nature Conservancy Stormwater Heatmap 

Documentation for the Stormwater Heatmap is in progress. Hydrology simulations were performed using [PyHSPF](https://github.com/djlampert/PyHSPF). Watershed factors used in this simulation were selected based on stormwater modeling guidance from the Department of Ecology. See the [Western Washington Hydrology Model Users Manual for more information.](https://fortress.wa.gov/ecy/madcap/wq/2014SWMMWWinteractive/2014%20SWMMWW.htm#Topics/VolumeIII2014/VolIII%20AppB%202014.htm%3FTocPath%3D2014%2520SWMMWW%7CVolume%2520III%2520-%2520Hydrologic%2520Analysis%2520and%2520Flow%2520Control%2520BMPs%7C)

###**Limitations**


**Results not calibrated**
> Hydrology simulations were performed on a hydrologic response unit basins using regional parameters. Parameters have not been calibrated to actual data. A model-based gridded precipitation dataset was used (see [Mauger et al 2018](https://cig.uw.edu/news-and-events/publications/new-projections-of-changing-heavy-precipitation-in-king-county/) ). 


**Model does not include routing or storage**
>Modeling was performed on a lumped parameter basis. Attenuation due to routing or storage is not reflected in the model. For large watersheds (greater than ~ 1 square mile) high flows may not be accurate. 

**Model does not include deep groundwater baseflow**
>The modeling results include all hspf flow paths (surface flow, interflow, and groundwater flow), however baseflow from deep groundwater is not included. Consistent with WDFW guidance, results in this notebook have assumed a minimum baseflow of 1 cfs. 
---



# Project set up and functions

Execute the cells below to set up the notebook. 
If cell throws an error, you can safely ignore. 

If prompted to restart the runtime, click ```restart runtime```


In [0]:
#@title Update Folium Library { vertical-output: true, display-mode: "form" }
#@markdown Please run this cell. Double click to show code.  

# folium update
!pip uninstall folium -y
!pip install folium
import folium
!pip install geopandas

In [0]:
#@title Global Functions { run: "auto", display-mode: "form" }
#@markdown Please run this cell. Double click to show code.  

#@title Import Libraries
import geopandas as gpd
import pandas as pd
import folium
import branca
import requests
import json
import math
from itertools import product

import numpy
import altair
import pandas_gbq
from google.colab import widgets
from datetime import datetime
import ee
from folium.features import GeoJson, GeoJsonTooltip

DEFAULT_PROJECT = "stormwaterheatmap-hydrology"
SQM_TO_ACRES = 4046.8564224
_VALID_SOIL_TYPES = {0: 'A/B', 1: 'C', 2: 'D'}
_VALID_LANDUSES = {0: 'forest', 1: 'pasture', 2: 'lawn', 5: 'impervious'}
_VALID_SLOPE_CLASSES = {0: 'flat', 1: 'moderate', 2: 'steep'}
ALL_HRUs = [
    f'hru{soil}{landuse}{slope}'
    for soil, landuse, slope in product(_VALID_SOIL_TYPES, _VALID_LANDUSES,
                                        _VALID_SLOPE_CLASSES)
    if (landuse != 5) or (landuse == 5 and soil == 2)
]

ALL_MONTHS = list(range(1, 13))

def _get_soil(hruname):
    stype = int(hruname[0])
    return _VALID_SOIL_TYPES[stype]

def _get_land_cover(hruname):
    lutype = int(hruname[1])
    return _VALID_LANDUSES[lutype]

def _get_slope(hruname):
    slopetype = int(hruname[2])
    return _VALID_SLOPE_CLASSES[slopetype]

# 'hru description': lambda df: df.apply(lambda row: (row['soil'])+", "+(row['land cover'])+", "+(row['slope']), axis=1),
def _hru_descr(hruname):
    return ', '.join([fxn(hruname) for fxn in (_get_soil, _get_land_cover, _get_slope)])

def get_HRUs_in_watershed(ee, watershed_feature, tile_scale=10, image_file=None):
    """
    Retrieve a list of HRUs within a watershed from

    Parameters
    ----------
    ee : Initialized Earth Engine object
    watershed_geom : shapely Polygon
        Shapely object representing the watershed
    tile_ecale : int (optional, default = 10)
        Scale to which the detail of the image file should be reduced
    image_file : str (optional)
        EE image name that contains the HRUs. When not provided, falls back to:
        "users/stormwaterheatmap/hruOut_fixed".

    Returns
    -------
    HRUs : dict
        Dictionary where the keys are the HRU and the values are the areas in
        square meters

    """
    if not image_file:
        image_file = "users/stormwaterheatmap/public/hrusJan2020Mode"
    
    hrus = ee.Image(image_file)
    area = ee.Image.pixelArea()
    img = ee.Image.cat(area,hrus)
    regionReduce = img.reduceRegion(
        ee.Reducer.sum().group(1, 'hru'),
        watershed_feature, 2
    ).get('groups').getInfo()
    df = pd.DataFrame(regionReduce).rename(columns={"sum": "sq.m"})
    return df


ALL_HRUs = [
    f'hru{soil}{landuse}{slope}'
    for soil, landuse, slope in product(_VALID_SOIL_TYPES, _VALID_LANDUSES,
                                        _VALID_SLOPE_CLASSES)
    if (landuse != 5) or (landuse == 5 and soil == 2)
]
ALL_MONTHS = list(range(1, 13))

def _get_soil(hruname):
    stype = int(hruname[0])
    return _VALID_SOIL_TYPES[stype]


def _get_land_cover(hruname):
    lutype = int(hruname[1])
    return _VALID_LANDUSES[lutype]


def _get_slope(hruname):
    slopetype = int(hruname[2])
    return _VALID_SLOPE_CLASSES[slopetype]

# 'hru description': lambda df: df.apply(lambda row: (row['soil'])+", "+(row['land cover'])+", "+(row['slope']), axis=1),
def _hru_descr(hruname):
    return ', '.join([fxn(hruname) for fxn in (_get_soil, _get_land_cover, _get_slope)])

def get_HRUs_in_watershed(ee, watershed_id, scale = 10, tile_scale=10, image_file=None):
    """
    Retrieve a list of HRUs within a watershed from

    Parameters
    ----------
    ee : Initialized Earth Engine object
    watershed_geom : shapely Polygon
        Shapely object representing the watershed
    tile_ecale : int (optional, default = 10)
        Scale to which the detail of the image file should be reduced
    image_file : str (optional)
        EE image name that contains the HRUs. When not provided, falls back to:
        "users/stormwaterheatmap/hruOut_fixed".

    Returns
    -------
    HRUs : dict
        Dictionary where the keys are the HRU and the values are the areas in
        square meters

    """
    if not image_file:
        image_file = "users/stormwaterheatmap/public/hrusJan2020Mode"
    
    hrus = ee.Image(image_file)
    area = ee.Image.pixelArea()
    img = ee.Image.cat(area,hrus)
    regionReduce = img.reduceRegion(
        ee.Reducer.sum().group(1, 'hru'),
        watershed, scale
    ).get('groups').getInfo()
    df = pd.DataFrame(regionReduce).rename(columns={"sum": "sq.m"})
    return df


def process_HRUs(df):
    hru_mapper = dict(zip(
        [40, 41, 42, 140, 141, 142, 240, 241, 242, 50, 51, 52, 150, 151, 152],
        [250, 251, 252, 250, 251, 252, 250, 251, 252, 250, 251, 252, 250, 251, 252]
    ))

    hru_df = (
        df.assign(hru=lambda df: df['hru'].replace(hru_mapper))
        .assign(hruname=lambda df: df['hru'].map(lambda x: f'{x:03d}'))
        .loc[lambda df: ~df['hruname'].str.contains('3')]
        .groupby(by=['hru', 'hruname']).sum()
        .reset_index()
        .assign(**{
            'acres': lambda df: df['sq.m'] / SQM_TO_ACRES,
            'soil': lambda df: df['hruname'].apply(_get_soil),
            'land cover': lambda df: df['hruname'].apply(_get_land_cover),
            'slope': lambda df: df['hruname'].apply(_get_slope),
            'hru description': lambda df: df['hruname'].apply(_hru_descr),
        })
    )
    return hru_df



def run_query(datadir=None, x=None, y=None):
    #Authenticate to bigquery
    #key_path = datadir / "stormwaterheatmap-hydrology-8308154a1232.json"
    pandas_gbq.context.project = "stormwaterheatmap-hydrology"
    pandas_gbq.context.credentials = service_account.Credentials.from_service_account_file(
        str(key_path))

    #Generate query statement
    gridquery = """
        with location as (
            SELECT
            st_contains(st_geogfromtext(wkt_geom),
                st_geogpoint(
                    {lon},
                    {lat}
                    ) ) as grid,
                    geohash
            FROM
            `stormwaterheatmap-hydrology.geometry.geohash` )
            select geohash from location
            where grid
        """.format(lon=x, lat=y)
        #run query to find precip grid geohash
    project_id = "stormwaterheatmap-hydrology"
    bqOutput = pd.read_gbq(gridquery, project_id=project_id)
    gridID = bqOutput.values[0][0]
    namesAreas = []
    for hru in hrustoQuery.keys():
        namesAreas.append(
        hru + "*" + str(hrustoQuery.get(hru))
        )
    quantstmt = """
    approx_quantiles( ' + '
    + '.join({namesAreas}) + ', '
    +str(numofQuantiles)+ ')
    AS Quantiles'
    """.format(namesAreas=namesAreas)
    #generate quantile query statement
    qry = """
        Select
        {selectstmt}
        FROM   `stormwaterheatmap-hydrology.gfdl.{grid_id}`
        WHERE  year BETWEEN {year0} AND    {yearN}
        AND    month IN {months}
        """.format(
            grid_id=gridID, selectstmt=quantstmt, year0=year0, yearN=yearN, months=months)
    #Run query
    bqOutput = pd.read_gbq(qry, project_id=project_id)
    return bqOutput


def process_quantiles(flow_quantiles, name):
    flows = numpy.array(flow_quantiles.Quantiles[0]) * 9.80962964e-6  # mm/hr*m2 to cfs
    #flows = numpy.delete(flows, -1)  # remove the 0 value
    n = (flows.shape[0]-1)
    probs = numpy.arange(0, 1+(1 / n), 1 / n)
    excd = (1 - probs).round(2)
    return pd.DataFrame({'Exceedance': excd, name: flows})



def get_geohash_of_point(centroid, project_id=None):
    if not project_id:
        project_id = DEFAULT_PROJECT

    sql = '''with test as (
    SELECT
    st_contains(st_geogfromtext(wkt_geom),
        st_geogpoint({lon},
            {lat}) ) as grid,
            geohash
    FROM
    `stormwaterheatmap-hydrology.geometry.geohash` )
    select geohash from test
    where grid
    '''.format(lon=centroid[0], lat=centroid[1])
    return pd.read_gbq(sql, project_id=project_id).geohash[0]


def get_flow_quantiles(hru_df, geohash, year0, yearN, nquantiles,
                       *months, project_id=None):
    if not project_id:
        project_id = DEFAULT_PROJECT

    names_and_areas = hru_df[['hruname', 'sq.m']].apply(
        lambda r: f"(hru{r['hruname']} * {r['sq.m']})", axis=1
    ).tolist()

    quantstmt = 'approx_quantiles(({names_areas}), {nquantiles}) AS Quantiles'.format(
        names_areas=' + '.join(names_and_areas),
        nquantiles=nquantiles
    )

    qry = """
    SELECT
    {selectstmt}
    FROM   `stormwaterheatmap-hydrology.gfdl.{grid_id}`
    WHERE  year BETWEEN {year0} AND {yearN}
    AND    month IN ({months})
    """.format(
        grid_id=geohash, selectstmt=quantstmt,
        year0=year0, yearN=yearN,
        months=(', '.join(str(x) for x in months))
    )
    return pd.read_gbq(qry, project_id=project_id)



def prob_plot(quants):
    ymax = math.ceil(quants['Qhigh'].max(0))
    qplot = altair.Chart(quants).transform_fold(
        ['Qlow', 'Qhigh']
    ).mark_line().encode(
        x=altair.X('Exceedance:Q', axis=altair.Axis(format='%')),
        y=altair.Y(
            'value:Q',
            scale=altair.Scale(
                type='log',
                domain=[1,ymax],
                clamp=True),
                axis=altair.Axis(
                    title = 'Discharge (cfs)'
                )),
        color='key:N',
        tooltip=['Exceedance:Q', 'value:Q'], 
        )
    return qplot


def hilo_flow_widgets(quants):
    low = (quants.loc[quants['Exceedance'] == 0.95]['Qlow']).values[0]
    high =round((quants.loc[quants['Exceedance'] == 0.10]['Qhigh']).values[0],2)

    low_label = widgets.Label(
        value=f'Low Flow: {low:.3g} cfs',
        description='Low Flow (cfs)',
    )

    high_label = widgets.Label(
        value=f'High Flow: {round(high,3)} cfs',
        description='Low Flow (cfs)',
    )

    return widgets.VBox([
        widgets.Label('Results'),
        low_label,
        high_label
    ])



    #add ee method to folium
    folium.Map.add_ee_layer = add_ee_layer

    # Set visualization parameters.
    LCvis_params = {'min': 0, 'max': 5,
                    "palette":["55775e","dacd7f","7e9e87","b3caff","844c8b","ead1ff"]};
    soils_vis = {'min': 0, 'max': 2,
                    "palette":['#564138', '#69995D', '#F06543']}
    slope_vis = {'min': 0, 'max': 2, 'dimensions': 400,
                    "palette": ['#009B9E', '#F1F1F1', '#C75DAB']}

    #Read watershed information from the watershed dictionary
    #center = watershed_dict["pour_point"]
    #watershed_gdf = GeoData(geo_dataframe=(watershed_dict["watershed_geometry"]), name='watershed')
    watershed = table3.filter(ee.Filter.eq("OBJECTID",2878))

    # Create a folium map object.
    eeMap = folium.Map(location=[center.y, center.x], zoom_start=13)

    #download stormwaterheatmap data from earth engine

    #landcover is a stormwaterheatmap asset:
    landcover = ee.Image("users/stormwaterheatmap/landcover_2m").clip(watershed)

    #soils data is a stormwaterheatmap asset:
    soils = ee.Image("users/stormwaterheatmap/soils2m").clip(watershed)

    #we generate slope information from the USGS National Elevation dataslet:
    NED = ee.Image("USGS/NED").clip(watershed)

    #classify by WWHM thresholds:
    thresholds = ee.Image([5.0, 15, 100]);
    slopeZone = ee.Terrain.slope(NED).gt(thresholds).reduce('sum');
    slope = slopeZone.clip(watershed)

    #add to the folium Map
    eeMap.add_ee_layer(ee, soils, soils_vis, 'Soils')
    eeMap.add_ee_layer(ee, slope, slope_vis, 'Slope')
    eeMap.add_ee_layer(ee, landcover, LCvis_params, 'Land cover')

    # Add a layer control panel to the map.
    eeMap.add_child(folium.LayerControl())
    return eeMap

def add_ee_layer(self, ee, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr=
        'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True).add_to(self)

%matplotlib inline

## Earth Engine Authentication

This notebook requires a Google Earth Engine Account to query and analyze spatial data. Visit https://earthengine.google.com/signup to sign up for an account.  

After you have registered your account. Run the `ee.Authenticate` function to authenticate your access to Earth Engine servers and `ee.Initialize` to initialize it. Upon running the following cell you'll be asked to grant Earth Engine access to your Google account. Follow the instructions printed to the cell. (Sometimes this takes awhile). 

In [0]:
ee.Authenticate() 
ee.Initialize()

## Big Query Authentication 
Likewise, you will need to authenticate to Big Query by running the cell below. 


In [0]:
#@title  { vertical-output: true, display-mode: "form" }
#@markdown Please run this cell. Double click to show code.  
project_id = "stormwaterheatmap-hydrology"
pandas_gbq.context.project = project_id
#test authentication by running a sample query
sql = """
SELECT geohash FROM `stormwaterheatmap-hydrology.geometry.geohash` LIMIT 1
"""
geohash_test_auth = pandas_gbq.read_gbq(
    sql,
    project_id=project_id
)

<style>
div.blue { background-color:#e6f0ff; border-radius: 5px; padding: 16px 20px 8px 20px;}
</style>




# Enter simulation parameters


In [0]:
#@title Input for Calculating Flow Quantiles { run: "auto", vertical-output: true, form-width: "500px", display-mode: "form" }
#@markdown Edit the variables below for your particular case:   
from google.colab import widgets

# Now we can create a grid, optional header_row and header_column
# control whether we want header elements in the grid

#@markdown **First Year of Simulation:** 
year0 = 1970 #@param {type:"slider", min:1970, max:2000, step:1}
#@markdown **Last Year of Simulation:**  
yearN = 2000  #@param {type:"slider", min:1970, max:2000, step:1}

#low flow is set for all months 
low_flow_months = ALL_MONTHS 
#@markdown **Months to use for High Flow Migration Period:** *```hard coded for now```*
#@markdown 
#@markdown ```January```

#@markdown **Months to use for Low Flow Migration Period:** *```hard coded for now```*  
#@markdown 
#@markdown ```January-December```

#high flow is set for upstream migration  
high_flow_months = list(range(1,2)) #high flow is calculated for January only 
#@markdown **Number of Quantile Values to Calculate:** 

numofQuantiles = 100 #@param {type:"integer"}

# Site Selection
Next, genearate a map of the watersheds. Since some libraries are limited in colab, the way to select a site from the map is to find the watershed ID, ```AU_ID``` from the map and enter it as text. This method is being used to demonstrate the quantile calcuation functions. 





In [0]:
#@title Display and choose a watershed { run: "auto", form-width: "400px", display-mode: "form" }
#@markdown This notebook can use any watershed feature class in  ```.geojson``` format. For now, use the geojson uploaded to github. 

#@markdown The default watersheds are from 
#@markdown Ecology's [Puget Sound Watershed Characterization Project](https://fortress.wa.gov/ecy/coastalatlas/wc/landingpage.html)

#@markdown **Enter URL below** (or leave as-is to accept the default): 
watershed_fileName = "https://gist.githubusercontent.com/wlrddatateam/5d9ecd7eb730fda17c24b7d9e1e65abf/raw/3363910c5e4d536d74fa55e560bf15cf07f8accc/CherryCreek_20200513_Feature.geojson" #@param {type:"string"}
idField = "AU_ID" #@param {type:"string"}
crs = "EPSG:4326"#@param {type:"string"}
response = requests.get(watershed_fileName)
data = response.json()
sheds = gpd.GeoDataFrame.from_features(
    data, crs=crs)
sheds["Colab_ID"]= sheds[idField].astype(str)
sheds = sheds.set_index("Colab_ID",drop=False)

# Create a folium map object.
folium.Map(prefer_canvas=True)
m = folium.Map(location=[47.3903, -122.0454], zoom_start=10, height=500)

#add tooltip
tooltip = GeoJsonTooltip(
    fields=["Colab_ID"],
    aliases=["WatershedID"],
    sticky=False,
    labels=True,
    style="""
        background-color: #F0EFEF;
        border: 2px solid black;
        border-radius: 3px;
        box-shadow: 3px;
    """
)
g = folium.GeoJson(
    sheds,
    tooltip = tooltip
).add_to(m)

m

In [0]:
#type in AU ID
#@title Enter Selected Watershed AU ID { run: "auto", display-mode: "form" }
#@markdown From the above map, enter the watershed ID (AU ID) in the field below: 
WatershedID = 	"6505"  #@param {type: "string"}
 
aoi = sheds.loc[WatershedID,"geometry"]
pd.DataFrame(sheds.loc[WatershedID,:]).drop("geometry", axis=0)

eeWS = gpd.GeoSeries([aoi]).__geo_interface__
#list of coordinates to pass to earth engine
eegeometry = list(eeWS['features'][0]['geometry']['coordinates'][0]) 
watershed = ee.Algorithms.GeometryConstructors.Polygon(eegeometry).getInfo()
HRUs = get_HRUs_in_watershed(ee, watershed)

def detail_map(watershed, ee=None):
    #add ee method to folium
    folium.Map.add_ee_layer = add_ee_layer

    # Set visualization parameters.
    LCvis_params = {'min': 0, 'max': 5,
                    "palette":["55775e","dacd7f","7e9e87","b3caff","844c8b","ead1ff"]};
    soils_vis = {'min': 0, 'max': 2,
                    "palette":['#564138', '#69995D', '#F06543']}
    slope_vis = {'min': 0, 'max': 2, 'dimensions': 400,
                    "palette": ['#009B9E', '#F1F1F1', '#C75DAB']}

    #Read watershed information from the watershed dictionary
    center =  ee.Geometry(watershed).centroid()
    

    # Create a folium map object.
    eeMap = folium.Map(location=[center.y, center.x], zoom_start=13)

    #download stormwaterheatmap data from earth engine

    #landcover is a stormwaterheatmap asset:
    landcover = ee.Image("users/stormwaterheatmap/landcover_2m").clip(watershed)

    #soils data is a stormwaterheatmap asset:
    soils = ee.Image("users/stormwaterheatmap/soils2m").clip(watershed)

    #we generate slope information from the USGS National Elevation dataslet:
    NED = ee.Image("USGS/NED").clip(watershed)

    #classify by WWHM thresholds:
    thresholds = ee.Image([5.0, 15, 100]);
    slopeZone = ee.Terrain.slope(NED).gt(thresholds).reduce('sum');
    slope = slopeZone.clip(watershed)

    #add to the folium Map
    eeMap.add_ee_layer(ee, soils, soils_vis, 'Soils')
    eeMap.add_ee_layer(ee, slope, slope_vis, 'Slope')
    eeMap.add_ee_layer(ee, landcover, LCvis_params, 'Land cover')

    # Add a layer control panel to the map.
    eeMap.add_child(folium.LayerControl())
    return eeMap

folium.Map.add_ee_layer = add_ee_layer

# Set visualization parameters.
LCvis_params = {'min': 0, 'max': 5,
                    "palette":["55775e","dacd7f","7e9e87","b3caff","844c8b","ead1ff"]};
soils_vis = {'min': 0, 'max': 2,
                    "palette":['#564138', '#69995D', '#F06543']}
slope_vis = {'min': 0, 'max': 2, 'dimensions': 400,
                    "palette": ['#009B9E', '#F1F1F1', '#C75DAB']}

    #Read watershed information from the watershed dictionary
center_obj =  ee.Geometry(watershed).centroid().getInfo()
center = center_obj.get('coordinates')
# Create a folium map object.
eeMap = folium.Map(location=[center[1], center[0]], zoom_start=13)

    #download stormwaterheatmap data from earth engine

    #landcover is a stormwaterheatmap asset:
landcover = ee.Image("users/stormwaterheatmap/landcover_2m").clip(watershed)

    #soils data is a stormwaterheatmap asset:
soils = ee.Image("users/stormwaterheatmap/soils2m").clip(watershed)

    #we generate slope information from the USGS National Elevation dataslet:
NED = ee.Image("USGS/NED").clip(watershed)

#classify by WWHM thresholds:
thresholds = ee.Image([5.0, 15, 100]);
slopeZone = ee.Terrain.slope(NED).gt(thresholds).reduce('sum');
slope = slopeZone.clip(watershed)

    #add to the folium Map
eeMap.add_ee_layer(ee, soils, soils_vis, 'Soils')
eeMap.add_ee_layer(ee, slope, slope_vis, 'Slope')
eeMap.add_ee_layer(ee, landcover, LCvis_params, 'Land cover')

    # Add a layer control panel to the map.
eeMap.add_child(folium.LayerControl())


  #@title
def hru_barchart(hru_df):
    chart = altair.Chart(hru_df)
    c1 = chart.mark_text().encode(
        y=altair.Y('hru description', axis=None)

    )
    c4 = chart.mark_bar().encode(
        x =altair.X('acres:Q'),
        y=altair.Y('hru description:N', sort=altair.EncodingSortField(field='acres', order='descending')),
        color = 'hru description:N'
        #color='land use'
    ).properties(width=300)
    #desc = c1.encode(text='hru description:N').properties(title='descriptions')
    #ac = c1.encode(text='acres:Q').properties(title='acres')
    #tab = altair.hconcat(desc, ac)
    return c4

def process_HRUs(df):
    hru_df = (
        #df.assign(hru=lambda df: df['hru'].replace(hru_mapper))
        df.assign(hruname=lambda df: df['hru'].map(lambda x: f'{x:03d}'))
        #.loc[lambda df: ~df['hruname'].str.contains('3')]
        .groupby(by=['hru', 'hruname']).sum()
        .reset_index()
        .assign(**{
            'acres': lambda df: df['sq.m'] / SQM_TO_ACRES,
            'soil': lambda df: df['hruname'].apply(_get_soil),
            'land cover': lambda df: df['hruname'].apply(_get_land_cover),
            'slope': lambda df: df['hruname'].apply(_get_slope),
            'hru description': lambda df: df['hruname'].apply(_hru_descr),
        })
    )
    return hru_df

HRUs = HRUs.assign(hruname=lambda df: df['hru'].map(lambda x: f'{x:03d}'))
#HRUs.loc[lambda df: ~df['hruname'].str.contains('3')]
HRUs.drop(HRUs[HRUs.hruname.str[1] == "9"].index,inplace=True)
HRUs.drop(HRUs[HRUs.hruname == "255"].index,inplace=True)
HRUs= process_HRUs(HRUs)

centroid = ee.Geometry(watershed).centroid().getInfo().get('coordinates')
geohash = get_geohash_of_point(centroid, project_id=project_id)


low_flow_quants = get_flow_quantiles(HRUs, geohash, year0, yearN, numofQuantiles, *low_flow_months)
high_flow_quants = get_flow_quantiles(HRUs, geohash, year0, yearN, numofQuantiles, *high_flow_months)

quants = process_quantiles(low_flow_quants, 'Qlow').merge(
    process_quantiles(high_flow_quants, 'Qhigh'), on=['Exceedance']
)

qplot = prob_plot(quants)

summary_table = quants.style.apply(
    lambda x: ["background: #1e88e5" if x['Exceedance']==0.10  else("background: #F58518" if x['Exceedance']==0.95 else "") for v in x], axis = 1).hide_index()

# Display Results


In [0]:
#@title  { run: "auto",  form-width: "400px", display-mode: "form" }
#rounding 
decimals = pd.Series([2, 2, 2], index = ['Exceedance','Qlow','Qhigh'])
high = quants.round(decimals)[(quants.Exceedance == 0.1)]["Qhigh"] 
high = high.values[0]
low = quants.round(decimals)[(quants.Exceedance == 0.95)]["Qlow"] 
low = low.values[0]
lowflag = ""
if low < 1.0:
  low = 1.0 
  lowflag = "   Modeled low-flow below 1.0 cfs threshold."
%load_ext google.colab.data_table
from IPython.core.display import display, HTML

html_report = f'''
  <style type="text/css">
  div.test
  {{
  padding: 4px;
  border-top-style: none;
  border-right-style: none;
  border-bottom-style: none;
  border-left-style: solid;
  border-color: #333333; 
  margin: 0px 0px 0px 50px;
  }}
  </style>
  <div class="test">
  <b> High Passage Flow:</b> {high} cfs
  <br> 
  <b>Low Passage Flow:</b> {low} cfs.
  <br> 
  <br> <em>{lowflag}</em>
     '''
t = widgets.TabBar(["Land use summary","Hydrology Summary","Map"])

## Land use summary - [0]
with t.output_to(0):
  grid = widgets.Grid(1, 2) 

  with grid.output_to(0,0):
       display(hru_barchart(HRUs).properties(
    title='Hydrologic Response Units'))
       
  with grid.output_to(0,1):
       display(HRUs)

## Hydrology Summary - [1]
with t.output_to(1):
  grid2 = widgets.Grid(1, 2) 
  with grid2.output_to(0,0):
    display(qplot.mark_line() + qplot.mark_point().properties(
    title='Flow Duration Curve'
    ))
    display(HTML(html_report))
  with grid2.output_to(0,1):
    display((quants.round(decimals)))
    

## Map - [2]
with t.output_to(2):
  display(eeMap)
  

#GRID --- 

  
  